In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import CohereEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatCohere
from langchain.callbacks.base import BaseCallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from dotenv import load_dotenv
import os
load_dotenv()
COHERE_API_KEY = os.getenv('COHERE_API_KEY')

TypeError: issubclass() arg 1 must be a class

In [ ]:
# loader = CSVLoader(file_path="C:/Users/jayit/GlobeHopper/source/data/output_cleaned.csv", encoding="utf-8")
# documents = loader.load()

In [ ]:
# documents

In [ ]:
# len(documents)

In [ ]:
embeddings = CohereEmbeddings(cohere_api_key=COHERE_API_KEY)
# db = Chroma.from_documents(documents, embeddings, persist_directory="./chroma_db_image")
db = Chroma(persist_directory="./chroma_db_image", embedding_function=embeddings)

In [ ]:
retriever = db.as_retriever()

In [ ]:
model = ChatCohere(
    streaming=True,
    callback_manager=BaseCallbackManager([
        StreamingStdOutCallbackHandler()
    ]),
    )

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
template = """Fetch hotel names, hotel rating, address, attractions(if any), description, hotel facilities, 
map, phone number, pincode, website url, images below details based only on the following context,
if you don't know the answer just say I don't know, don't try to make up:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
print(chain.invoke("recommend hotel names in Sarande Albania"))

In [ ]:
print(chain.invoke("recommend hotel names in Sarande Albania"))

In [ ]:
print(chain.invoke("recommend best resorts in Alleppey, Kerala"))

In [ ]:
print(chain.invoke("recommend best resorts in Alleppey, Kerala"))

In [ ]:
print(chain.invoke("What are the amenities of Oyo 16360 Green Homes?"))

In [ ]:
print(chain.invoke("What are the ammenities of Oyo 16360 Green Homes in Patna Bihar?"))

In [ ]:
print(chain.invoke("Fetch the booking website for Dolphin Hostel"))

In [ ]:
print(chain.invoke("Fetch the booking website for Hostel Vila 3 in Berat, Albania"))